In [19]:
import pandas as pd
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
%pip install kaleido

In [2]:
# Source data: https://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo.html

In [179]:
df = pd.read_csv('data/spline_merged_ice_core_yearly.csv',names=['date','co2'])

In [4]:

def parse_dt(dt):
    if dt < 100:
        dt = '00' + str(dt)
    elif dt < 1000:
        dt = '0' + str(dt)
    else:
        dt = str(dt)
    return datetime.strptime(dt, '%Y.%m')

In [183]:
emissions_1700 = emissions[emissions.year >= 1700]

emissions_1700.index = pd.to_datetime(emissions_1700.date)

In [197]:
emissions_pre1960 = emissions_1700[emissions_1700.year <= 1960]
mean_pre1960 = (emissions_pre1960.co2.iloc[-1] - emissions_pre1960.co2.iloc[0]) / (relativedelta(emissions_pre1960.date.iloc[-1], emissions_pre1960.date.iloc[0]).years)

In [198]:
emissions_post1960 = emissions_1700[emissions_1700.year > 1960]
mean_post1960 = (emissions_post1960.co2.iloc[-1] - emissions_post1960.co2.iloc[0]) / (relativedelta(emissions_post1960.date.iloc[-1], emissions_post1960.date.iloc[0]).years)

In [239]:
keeling_trace = go.Scatter(
    x=emissions_1700['date'], 
    y=emissions_1700['co2'],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='teal', 
        color=emissions_1700.co2.rolling(window='365D').apply(lambda x: x.iloc[-1] - x.iloc[0]).clip(lower=0.0),
        showscale=True,
        colorbar=dict(
            orientation='h',
            ticksuffix=' ppm/yr',
            y=0.65,
            x=0.39,
            len=0.7,
            thickness=10.0
        )
    ),
)

title="""
Atmospheric CO2 record based on ice core data before 1958, and
<br>yearly averages of direct observations from Mauna Loa and the South Pole after and including 1958.
"""

layout = go.Layout(
    title=dict(text=title, x=0.05, y=0.9),
    font_family='serif',
    title_font_size=15,
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        showline=True,
        ticklen=10,
        tickcolor='black',
        ticks='outside',
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        title='CO2',
        side='right',        
        ticklen=10,
        tickcolor='black',
        ticks='outside',
        ticksuffix=' ppm  '
    ),
    margin=dict(l=20, r=120, t=30, b=20),
    width=1300,
    height=600,
)

fig = go.Figure(data=keeling_trace, layout=layout)

x_to_dt = lambda d: datetime.strptime(d, '%Y.%m')

fig.add_shape(type='line',
    xref='x', yref='y',
    x0=x_to_dt('1705.1'),
    x1=x_to_dt('2020.1'),
    y0=316, 
    y1=316,
    line=dict(
        color='black',
        dash='dot',
        width=1,
    )
)

float_formatter = "{:.2f} CO&#8322; / year".format

fig.add_annotation(
    font=dict(size=15),
    x=x_to_dt('1710.1'),
    y=300,
    showarrow=False,
    text='Average emissions < 1960<br>' + float_formatter(mean_pre1960),
    textangle=0,
    xanchor='left')

fig.add_annotation(
    font=dict(size=15),
    x=x_to_dt('1710.1'),
    y=330,
    showarrow=False,
    text='Average emissions > 1960<br>' + float_formatter(mean_post1960),
    textangle=0,
    xanchor='left')

fig.show()


In [242]:
fig.write_html("output/keeling.html")
fig.write_image("output/keeling.png")